In [1]:
import sqlalchemy as sa
import psycopg2
import pandas as pd
import shapely.wkt
import geopandas as gpd

In [2]:
from app.core.config import get_settings
from sqlalchemy import create_engine

settings = get_settings()
engine = sa.create_engine(settings.db.url.get_secret_value().replace("asyncpg", "psycopg2"),
                          future=True)

In [9]:
import pydeck as pdk
import geopandas as gpd
from sqlalchemy import text

q = "SELECT geom, tags FROM osm LIMIT 2"
    



,geom,tags
0,"LINESTRING (-122.77668 37.60303, -122.77669 37...",{'name': 'United States of America (Pacific Oc...
1,"LINESTRING (-121.49454 35.98944, -121.49468 35...","{'ref': 'CA 1', 'name': 'Cabrillo Highway', 'l..."


In [12]:
from sqlalchemy.sql import Executable

sqlalchemy.sql.elements.TextClause

In [57]:
from typing import Union, cast, Dict, Any, Generator
from sqlalchemy.engine import Connection, Engine
from sqlalchemy.sql.expression import Executable
from pydeck.data_utils.viewport_helpers import compute_view, bbox_to_zoom_level

from typing import Optional, Union, cast, Dict, Any, Generator


from pydeck.data_utils import compute_view

def geodf_to_viewstate(df: gpd.GeoDataFrame) -> pdk.ViewState:
    bbox = df.total_bounds
    center_lat = (bbox[1] + bbox[3]) / 2
    center_lng = (bbox[0] + bbox[2]) / 2
    zoom_level = bbox_to_zoom_level([list(bbox[:2]), list(bbox[2:])])
    return pdk.ViewState(center_lng, center_lat, zoom_level=zoom_level)

def view_gpd(df: gpd.GeoDataFrame, **kwargs) -> pdk.Deck:
    return pdk.Deck(layers=[pdk.Layer("GeoJsonLayer", df)], 
                    initial_view_state=geodf_to_viewstate(df),
                    **kwargs)

class OsmDb:
    
    def __init__(self, engine: Engine) -> None:
        self._engine = engine

    def run(self, query: Union[str, Executable], **kwargs: Dict[str, Any]) -> Union[gpd.GeoDataFrame, Generator[gpd.GeoDataFrame, None, None]]:
        if isinstance(query, str):
            query = text(query)
        with self._engine.begin() as conn:
            df = gpd.read_postgis(cast(Executable, query), conn, **kwargs)
        return df
    
    def view(self, query: Union[str, Executable], layer_opts: Optional[Dict[str, Any]] = None, **kwargs: Dict[str, Any]) -> pdk.Deck:
        df = self.run(query, **kwargs)
        assert isinstance(df, gpd.GeoDataFrame)
        initial_view_state = geodf_to_viewstate(df)
        return pdk.Deck(layers=[pdk.Layer("GeoJsonLayer", df)], initial_view_state=initial_view_state, **(layer_opts or {}))




In [54]:
df = osm.run("""
    select st_buildarea(geom) as geom, tags->>'name' AS name
    from osm w
    where category = 'boundary' 
    and tags->>'name' = 'Oakland' 
    and tags->>'boundary' = 'administrative' 
    and tags->>'admin_level'= '8' 
    limit 1
    """)

In [65]:
osm.run(""" 
    WITH trajectors AS (
        SELECT *
        FROM osm
        WHERE (
            (category='boundary' AND ST_BuildArea(geom) IS NOT NULL)
            OR GeometryType(geom) = 'Polygon')
    )
    SELECT * FROM trajectors
    LIMIT 2
""")

,osm_type,osm_id,attrs,tags,geom,category,fts
0,R,2778884,{},"{'name': 'Silver Peak Wilderness', 'type': 'bo...","MULTILINESTRING ((-121.43896 35.89099, -121.43...",boundary,"'1b':26 'area':11,31 'boundari':5 'forest':15 ..."
1,R,5366706,{},"{'name': 'Owens Peak Wilderness', 'type': 'mul...","MULTILINESTRING ((-118.05696 35.70358, -118.05...",boundary,"'1b':19 'area':11,22 'blm':13 'multipolygon':5..."


In [66]:
import geopy

In [67]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="mercator")

In [75]:
location = geolocator.geocode("United States", geometry="geojson")
print(location)

United States


In [79]:
# Rules for areas
# https://github.com/carolinlawrence/overpass-nlmaps/blob/master/rules/areas.osm3s

AttributeError: 'dict' object has no attribute 'place_id'